In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import numpy as np
import os
import time

from Software.GenuVP import runGNVP as gnvp

from PlaneDefinition.plane import Airplane as Plane
from PlaneDefinition.wing import Wing as wg
import PlaneDefinition.wing as wing

from Database.getresults import Database_2D
from Database import DB3D , BASEGNVP
from Visualization import plotting as aplt

In [ ]:
HOMEDIR = os.getcwd()

# Airfoil Data

In [ ]:
db = Database_2D(HOMEDIR)
airfoils = db.getAirfoils()
polars = db.Data

# Get Plane

In [ ]:
Origin = np.array([0, 0, 0])

In [ ]:
wingPos = np.array([0,0,0])
wingOrientation = np.array([0,0,0])

mainWing = wg(name="wing",
              airfoil=airfoils['NACA4415'],
              Origin=Origin + wingPos,
              Orientation=wingOrientation,           
              isSymmetric=True,
              span=2 * 1.130,
              sweepOffset=0,
              dihAngle=0,
              chordFun=wing.linearChord,
              chord=[0.159, 0.072],
              spanFun=wing.linSpan)
# mainWing.plotWing()

In [ ]:
elevatorPos = np.array([0.54,0,0])
elevatorOrientantion = np.array([0,0,0])

elevator = wg(name = "tail",
              airfoil=airfoils['NACA0008'],
              Origin=Origin+ elevatorPos,
              Orientation=elevatorOrientantion,
              isSymmetric=True,
              span= 2 * 0.169,
              sweepOffset=0,
              dihAngle=0,
              chordFun=wing.linearChord,
              chord=[0.130, 0.03],
              spanFun=wing.linSpan)
# elevator.plotWing()

In [ ]:
rudderPos = np.array([0.54, 0, 0.1])
rudderOrientantion = np.array([0,0,90])

rudder = wg(name= "rudder",
            airfoil=airfoils['NACA0008'],
            Origin=Origin + elevatorPos,
            Orientation= rudderOrientantion,
            isSymmetric=False,
            span= 0.169,
            sweepOffset=0,
            dihAngle=0,
            chordFun=wing.linearChord,
            chord=[0.130, 0.03],
            spanFun=wing.linSpan)
# rudder.plotWing()


In [ ]:
liftingSurfaces = [mainWing, elevator, rudder]

ap = Plane("Plane",liftingSurfaces)
ap.accessDB(HOMEDIR,DB3D)
# ap.visAirplane()

In [ ]:
cleaning = False
calcGenu = True
calcBatchGenu = True

## Batch Run

In [ ]:
AoAmax = 15
AoAmin = -6
NoAoA = (AoAmax - AoAmin) + 1
angles = np.linspace(AoAmin, AoAmax, NoAoA)

Uinf = 20

In [ ]:
ap.batchangles(angles)
if calcBatchGenu == True:
    genuBatchArgs = [ap, BASEGNVP, polars,"Xfoil",Uinf, angles]
    ap.runSolver(gnvp.runGNVP,genuBatchArgs)
